In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [10]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',input_shape=(32, 32, 3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001,))

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001,))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001,))


#卷積組合
classifier.add(Convolution2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001,))


#卷積組合
classifier.add(Convolution2D(512, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001,))

classifier.add(Dropout(0.25))

# #flatten
classifier.add(Flatten())
# #FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu
classifier.add(Dropout(0.5))
# #輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

# #超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/50
50000/50000 [==============================] - 12s 234us/step - loss: 2.1068 - acc: 0.2049
Epoch 2/50
50000/50000 [==============================] - 10s 201us/step - loss: 1.9037 - acc: 0.2543
Epoch 3/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.7794 - acc: 0.2972
Epoch 4/50
50000/50000 [==============================] - 10s 199us/step - loss: 1.7387 - acc: 0.3197
Epoch 5/50
50000/50000 [==============================] - 10s 202us/step - loss: 1.6891 - acc: 0.3345
Epoch 6/50
50000/50000 [==============================] - 10s 199us/step - loss: 1.5892 - acc: 0.3919
Epoch 7/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.4665 - acc: 0.4495
Epoch 8/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.3372 - acc: 0.5172
Epoch 9/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.2146 - acc: 0.5727
Epoch 10/50
50000/50000 [==============================] - 10s 198us/step - loss: 

In [11]:
scores = classifier.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 173us/step
Test loss: 1.1808536009788513
Test accuracy: 0.8139
